In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
from datetime import datetime

In [ ]:
query_list = ["풍년", "흉년", "태풍"]
df_list = list() # 각 검색어에 대한 데이터프레임이 들어갈 리스트 

sort = 1
s_date = "2016.01.01"
e_date = "2020.09.28"
s_from = "20160101"
e_to = "20200928"

mycolumns = ['date', 'press','title','content','link']

for query in query_list : 
  num = 1 # 먼저 첫 400페이지는 그냥 긁어와야겠죠

  while True :  # 어떤 조건을 만족하기 전까지 반복을 끝내지 않습니다.
      page = 1
      
      if num >= 2 : # 400페이지를 긁었다면, 그 이후부터는 아래의 코드를 진행합니다.
          year = int(date_list[-1][0:4] + date_list[-1][5:7] + date_list[-1][8:10]) #YYYYMMDD형태
          # 먼저, 마지막 400페이지에 있는 날짜를 가져옵니다. 이 날짜를 시작으로 다시 검색할 것입니다. 
      
          if year <= 20151231 : # 우리가 원하는 날짜 전의 날짜가 나오면, break해서 종료해줍니다.
              break
              
          s_date, s_from = '2016.01.01', '20160101' # 시작날짜는 동일하게 원했던 날짜로 해주고 
          e_date, e_to = date_list[-1][:-1], str(year) # 종료날짜는 방금 전 시도까지 했었던 마지막페이지의 날짜로 해줍니다.
          
      title_list = [] # 제목 모음 리스트
      content_list = [] # 내용 모음 리스트
      link_list = [] # 링크 모음 리스트
      date_list = [] # 날짜 모음 리스트
      press_list = [] # 언론사 모음 리스트
      
      while page <= 3991 : # 400페이지까지 돌아줍니다. 
          url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + query + "&sort="+str(sort)+"photo=0&field=0&pd=3&ds=" + s_date + "&de=" + e_date + "&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:dd,p:from" + s_from + "to" + e_to +",a:all&start=" + str(page)
          response = requests.get(url)
          html = response.text
      
          soup = BeautifulSoup(html,'html.parser')
          li_tmp = soup.find_all('li',{'class' : 'bx'})
      
          for i in li_tmp : 
              try : 
                  title = i.find('a',{'class' : 'news_tit'}).text #제목
                  content = i.find('div',{'class' : 'news_dsc'}).text #내용
                  link = i.find('a',{'class' : 'news_tit'})['href'] #링크
                  press = i.find('a',{'class' : 'info press'}).text #언론사
              
                  date_set = i.findAll('span',{'class' : 'info'})#날짜
                  if date_set : 
                      if 'A' in date_set[0].text or '면' in date_set[0].text :
                          date = date_set[1].text
                      else : 
                          date = date_set[0].text
          
                  title_list.append(title)
                  content_list.append(content)
                  link_list.append(link)
                  press_list.append(press)
                  date_list.append(date)
          
              except : 
                  continue
      
          page += 10
          
      mydata = {
      'date' : date_list,
      'press' : press_list,
      'title' : title_list,
      'content' : content_list,
      'link' : link_list
      }
      
      my_df = pd.DataFrame(data = mydata, columns = mycolumns)

    df_list.append(my_df)

In [ ]:
final_list = list()
for my_df in df_list : 
  my_df = my_df.drop_duplicates("content")
  final_list.append(my_df)

,date,press,title,content,link
0,2020.02.26.,새전북신문,"생진원, 민간주도형 사업 참여기업간 상생협력 MOU추진","대풍년영농조합법인(대표 서형원), 두손푸드(대표 구성규) 등 4개사 대표와 임원...",http://sjbnews.com/news/news.php?number=677709
1,2020.02.26.,전북도민일보,"생진원, 민간주도 사업 참여기업간 상생협력 업무협약","이번 업무협약식은 생진원 김동수 원장을 비롯해 사업 참여기업 만선영어영농조합, ...",http://www.domin.co.kr/news/articleView.html?i...
2,2020.02.26.,전라일보,"전북생진원, 민간주도형 사업 참여기업간 상생협력 MOU 추진","이날 만선영어영농조합(대표 차호식)과 청맥㈜(대표 김재주), 대풍년영농조합법인(...",http://www.jeollailbo.com/news/articleView.htm...
3,2020.02.26.,시민일보,"영암군, 벼 병해충 공동방제 지원","및 경영비 일부를 절감시켜 농가소득을 증대시키기 위해 최선을 다하겠다""며 ""벼멸...",http://www.siminilbo.co.kr/news/newsview.php?n...
4,2020.02.26.,아시아경제,[고규홍의 나무와 사람] 한라山神은 왜 산천단으로 내려왔을까,"바람이 불든 눈이 오든, 풍년이든 흉년이든 제주 사람들은 살아있다는 것만으로도 ...",https://view.asiae.co.kr/article/2020022616582...
...,...,...,...,...,...
3994,2019.09.13.,미주중앙일보,"MLB, 약물 능가하는 '탱탱볼' 시대",팀도 개인도 홈런 풍년이다. 시애틀 매리너스는 개막전을 시작으로 20경기 연속 ...,http://www.koreadaily.com/news/read.asp?art_id...
3995,2019.09.13.,국민일보,"‘신이 내린 팔자’ 장항준 감독, “아내 김은희 작가 덕에 먹고 산다”","그는 1995년 장 감독과 선·후배로 만나 연인으로 발전, 1998년 부부의 연...",http://news.kmib.co.kr/article/view.asp?arcid=...
3996,2019.09.13.,농기자재신문,가요무대… 풍년가로 가을 분위기를 물씬 풍겼다,공연 초반에 국악인 출신의 정수빈은 경기민요인 ‘풍년가’로 가을 분위기를 물씬 ...,http://www.newsam.co.kr/news/article.html?no=1...
3997,2019.09.13.,아시아경제,"'라디오스타' 윤종신 ""장항준 감독 부인 김은희 작가 재밌는 친구""","그는 지난 1995년 장 감독과 선·후배로 만나 연인으로 발전, 1998년 결혼...",https://view.asiae.co.kr/article/2019091314391...
